In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from openpyxl.workbook import Workbook

In [17]:
import pandas as pd

In [137]:
import time
import datetime

In [138]:
import smtplib
from email.mime.multipart import MIMEMultipart

In [139]:
browser = webdriver.Chrome('/Users/mac/Downloads/chromedriver 4') 

In [21]:
#Setting ticket types paths
return_ticket = "//label[@id='flight-type-roundtrip-label-hp-flight']"
one_way_ticket = "//label[@id='flight-type-one-way-label-hp-flight']"
multi_ticket = "//label[@id='flight-type-multi-dest-label-hp-flight']"

In [140]:
def ticket_chooser(ticket):
    try:
        ticket_type = browser.find_element_by_xpath(ticket)
        ticket_type.click()
    except Exception as e:
        pass

In [141]:
def dep_country_chooser(dep_country):
    fly_from = browser.find_element_by_xpath("//input[@id='flight-origin-hp-flight']")
    time.sleep(1)
    fly_from.clear()
    time.sleep(1.5)
    fly_from.send_keys('  ' + dep_country)
    time.sleep(1.5)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1.5)
    first_item.click()

In [142]:
def arrival_country_chooser(arrival_country):
    fly_to = browser.find_element_by_xpath("//input[@id='flight-destination-hp-flight']")
    time.sleep(1)
    fly_to.clear()
    time.sleep(1.5)
    fly_to.send_keys('  ' + arrival_country)
    time.sleep(1.5)
    #first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    #time.sleep(1.5)
    #first_item.click()

In [143]:
def dep_date_chooser(month, day, year):
    dep_date_button = browser.find_element_by_xpath("//input[@id='flight-departing-hp-flight']")
    dep_date_button.clear()
    dep_date_button.send_keys(month + '/' + day + '/' + year)

In [144]:
def return_date_chooser(month, day, year):
    return_date_button = browser.find_element_by_xpath("//input[@id='flight-returning-hp-flight']")
    for i in range(11):
        return_date_button.send_keys(Keys.BACKSPACE)
    return_date_button.send_keys(month + '/' + day + '/' + year)

In [145]:
def search():
    search = browser.find_element_by_xpath("//button[@class='btn-primary btn-action gcw-submit']")
    search.click()
    time.sleep(15)
    print('Results ready!')

In [146]:
df = pd.DataFrame()
def compile_data():
    global df
    global dep_times_list
    global arr_times_list
    global airlines_list
    global price_list
    global durations_list
    global stops_list
    global layovers_list
    #departure times
    dep_times = browser.find_elements_by_xpath("//span[@data-test-id='departure-time']")
    dep_times_list = [value.text for value in dep_times]
    #arrival times
    arr_times = browser.find_elements_by_xpath("//span[@data-test-id='arrival-time']")
    arr_times_list = [value.text for value in arr_times]
    #airline name
    airlines = browser.find_elements_by_xpath("//span[@data-test-id='airline-name']")
    airlines_list = [value.text for value in airlines]
    #prices
    prices = browser.find_elements_by_xpath("//span[@data-test-id='listing-price-dollars']")
    price_list = [value.text for value in prices]
    #durations
    durations = browser.find_elements_by_xpath("//span[@data-test-id='duration']")
    durations_list = [value.text for value in durations]
    #stops
    stops = browser.find_elements_by_xpath("//span[@class='number-stops']")
    stops_list = [value.text for value in stops]
    #layovers
    layovers = browser.find_elements_by_xpath("//span[@data-test-id='layover-airport-stops']")
    layovers_list = [value.text for value in layovers]
    now = datetime.datetime.now()
    current_date = (str(now.year) + '-' + str(now.month) + '-' + str(now.day))
    current_time = (str(now.hour) + ':' + str(now.minute))
    current_price = 'price' + '(' + current_date + '---' + current_time + ')'
    for i in range(len(dep_times_list)):
        try:
            df.loc[i, 'departure_time'] = dep_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_time'] = arr_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'airline'] = airlines_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'duration'] = durations_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'stops'] = stops_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'layovers'] = layovers_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, str(current_price)] = price_list[i]
        except Exception as e:
            pass
    print('Excel Sheet Created!')

In [147]:
for i in range(8):    
    link = 'https://www.expedia.co.in/'
    browser.get(link)
    time.sleep(5)
    #choose flights only
    flights_only = browser.find_element_by_xpath("//button[@id='tab-flight-tab-hp']")
    flights_only.click()
    ticket_chooser(return_ticket)
    dep_country_chooser('New Delhi')
    arrival_country_chooser('Mumbai')
    dep_date_chooser('15', '08', '2019')
    return_date_chooser('20', '08', '2019')
    search()
    compile_data()
    #save values for email
    current_values = df.iloc[0]
    cheapest_dep_time = current_values[0]
    cheapest_arrival_time = current_values[1]
    cheapest_airline = current_values[2]
    cheapest_duration = current_values[3]
    cheapest_stops = current_values[4]
    cheapest_price = current_values[-1]
    print('run {} completed!'.format(i))
    df.to_excel('flights.xlsx')
    time.sleep(3600)

Results ready!
Excel Sheet Created!
run 0 completed!


KeyboardInterrupt: 

In [148]:
df.head(500)

,departure_time,arrival_time,airline,duration,stops,layovers,price(2019-8-4---10:43)
0,22:30,00:45,IndiGo,2h 15m,(Direct),2h 50m in AMD\nStopover 2h 50m in AMD,"₹5,343"
1,23:35,01:45,SpiceJet,2h 10m,(Direct),5h 50m in HYD\nStopover 5h 50m in HYD,"₹5,435"
2,20:30,22:45,SpiceJet,2h 15m,(Direct),9h 15m in HYD\nStopover 9h 15m in HYD,"₹5,435"
3,21:35,23:55,SpiceJet,2h 20m,(Direct),13h 25m in HYD\nStopover 13h 25m in HYD,"₹5,435"
4,19:50,21:55,Vistara,2h 5m,(Direct),20h 5m in ATQ\nStopover 20h 5m in ATQ,"₹5,503"
5,20:40,22:50,Vistara,2h 10m,(Direct),1h 30m in AMD\nStopover 1h 30m in AMD,"₹5,503"
6,21:30,23:40,Vistara,2h 10m,(Direct),4h 50m in AMD\nStopover 4h 50m in AMD,"₹5,503"
7,19:40,22:00,GoAir,2h 20m,(Direct),2h 25m in AMD\nStopover 2h 25m in AMD,"₹5,561"
8,20:50,23:10,GoAir,2h 20m,(Direct),4h in AMD\nStopover 4h in AMD,"₹5,561"
9,14:30,16:40,Vistara,2h 10m,(Direct),21h 35m in HYD\nStopover 21h 35m in HYD,"₹5,713"


In [ ]:
df.insert(loc=7, column='pred_price', value = 0 , allow_duplicates = False)


In [ ]:
df

In [ ]:
df.corr

In [135]:
a=datetime.datetime.now().time()

In [136]:
a

datetime.time(10, 40, 54, 113449)

In [129]:
b =  datetime.time(23,  59,59, 999999)


In [130]:
b

datetime.time(23, 59, 59, 999999)

In [132]:
if b>a:
    print ('sahil')

sahil


datetime.time(23, 50, 56, 291819)

In [ ]:
if a==b :
    link = 'https://www.expedia.co.in/'
    browser.get(link)
    time.sleep(5)
    #choose flights only
    flights_only = browser.find_element_by_xpath("//button[@id='tab-flight-tab-hp']")
    flights_only.click()
    ticket_chooser(return_ticket)
    dep_country_chooser('New Delhi')
    arrival_country_chooser('Mumbai')
    dep_date_chooser('15', '08', '2019')
    return_date_chooser('20', '08', '2019')
    search()
    compile_data()
    #save values for email
    current_values = df.iloc[0]
    cheapest_dep_time = current_values[0]
    cheapest_arrival_time = current_values[1]
    cheapest_airline = current_values[2]
    cheapest_duration = current_values[3]
    cheapest_stops = current_values[4]
    cheapest_price = current_values[-1]
    print('run {} completed!'.format(i))
    df.to_excel('flights.xlsx')
    
else:
    a=datetime.datetime.now().time()

In [ ]:
a=

In [107]:
while a>b:
    print('sahil')
else:
    a=datetime.datetime.now().time()

sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahi

sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahi

sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahi

sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahi

sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahi

...

sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahi

sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahi

sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahi

sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahil
sahi

KeyboardInterrupt: 

In [106]:
a

datetime.time(23, 58, 3, 817208)

In [104]:
b

datetime.time(23, 57, 57, 371454)

In [121]:
x=5
y=4

In [125]:
for i in range (30):
    a=datetime.datetime.now().time()
    while a==b :
        print('victory')
        time.sleep(86375)     
